# Making Redis Safer with per-lib seccomp filters

Putting a blanket seccomp filter over redis is good, but things could be better.

In [30]:
import pandas as pd
import numpy as np
import json

counts_df = pd.read_json('./stats/counts.json')

with open('./stats/missed.json') as f:
    parsed_data = json.loads(f.read())

missed_df = pd.DataFrame(list(parsed_data.items()), columns=['metric', 'value'])

missed_df

,metric,value
0,ringbuf_full,0
1,get_current_task_failed,0
2,get_parent_failed,0
3,get_pt_regs_failed,0
4,all_syscalls,21300757
5,relevant_syscalls,4218833
6,follow_ignore_pid,17081923
7,empty_stack_trace,0


In [27]:
counts_df.head()

,/usr/bin/redis-check-rdb,/usr/lib/x86_64-linux-gnu/ld-linux-x86-64.so.2,/usr/lib/x86_64-linux-gnu/libc.so.6,/usr/lib/x86_64-linux-gnu/libcap.so.2.66,/usr/lib/x86_64-linux-gnu/libcrypto.so.3,/usr/lib/x86_64-linux-gnu/libjemalloc.so.2,/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.33,FAILED
0,2001447.0,16.0,NaN,NaN,5.0,2.0,NaN,NaN
1,2000029.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
10,3.0,18.0,NaN,NaN,NaN,1.0,NaN,NaN
13,11.0,NaN,NaN,NaN,NaN,NaN,NaN,55.0
14,10.0,NaN,4.0,NaN,NaN,4.0,NaN,1.0


## Calculating Privilege Reduction

To calculate privilige reduction, a "danger score" needs to be assigned to each compartment. The privilige reduction can then be defined

$$
\%~\text{reduction} = \frac{\max(\text{library danger score})}{\text{whole application danger score}}
$$

### Danger score

- System calls are not all as dangerous as each other
  - An attacker having access to `getpid` has very little danger
  - An attacher having access to `execve` presents a lot of danger
  - The _danger score_ metric should reflect this

#### Three-tier system
- Assign the most dangerous syscalls a 3, middle danger a 2, safe a 1
- Use info from `../syscall-ranking`, taken from table 2, https://www.researchgate.net/publication/261959738_Using_Attack_Surface_Entry_Points_and_Reachability_Analysis_to_Assess_the_Risk_of_Software_Vulnerability_Exploitability, 2nd december 2024
- Here, `high-threat` is defined as taking full control of the system and `medium-threat` is defined denial of service.

In [26]:
with open("../syscall-ranking.yaml") as f:
    rankings = f.readlines()

scores = {}
v = 3
for line in rankings[3:]:
    l = line.strip()

    if not l or l[0] != "-":
        v = 2
        continue

    scores[int(l[2:])] = v


### Application Score

"Danger score" if a single seccomp filter to allow all system calls was applied to the application.

In [40]:
def danger_score(syscalls: pd.Series, scores: dict[int, int]) -> int:
    s = 0
    for n in syscalls:
        if n not in scores:
            s += 1
            continue
        s += scores[n]

    return s

danger_score(counts_df.index, scores)

54

### Library Scores

"Danger score" if a custom seccomp filter was implemented per library, along with the number of unique syscalls the library made.

In [124]:
def lib_to_syscalls(df: pd.DataFrame, col: str) -> list[int]:
    return df.index[pd.notna(df[col])].tolist()

def priv_red(danger, total):
    return 1 - (danger / total)

# priviliege reduction based on formula defined above

print(f"{'library':<50}: {'danger':^6} : {'count':^6} : {'priv. red. %':^10}")

total_danger = danger_score(counts_df.index, scores)

for col in counts_df.columns[:-1]:
    syscalls = lib_to_syscalls(counts_df, col)
    danger = danger_score(syscalls, scores)
    print(f"{col:<50}: {danger:^7}: {len(set(syscalls)):^7}:   {priv_red(danger, total_danger)*100:.1f}")    

print(f"{'all':<50}: {danger_score(counts_df.index, scores):^7}: {all_syscalls:^7}:   0.00")    


library                                           : danger : count  : priv. red. %
/usr/bin/redis-check-rdb                          :   34   :   33   :   37.0
/usr/lib/x86_64-linux-gnu/ld-linux-x86-64.so.2    :   15   :   15   :   72.2
/usr/lib/x86_64-linux-gnu/libc.so.6               :    3   :    3   :   94.4
/usr/lib/x86_64-linux-gnu/libcap.so.2.66          :    1   :    1   :   98.1
/usr/lib/x86_64-linux-gnu/libcrypto.so.3          :    7   :    7   :   87.0
/usr/lib/x86_64-linux-gnu/libjemalloc.so.2        :   16   :   14   :   70.4
/usr/lib/x86_64-linux-gnu/libstdc++.so.6.0.33     :    1   :    1   :   98.1
all                                               :   54   :   49   :   0.00


## Privilige Reduction

If one were to write a single blanket seccomp filter for the redis, it would include 49 syscalls*.

If one were to create a seccomp filter for each shared library, the most permissive filter would only allow 33 syscalls. This already shows a good degree of privilege reduction ($1-\frac{33}{49} = 32.7\%$).

Weighting each syscall by "danger" shows a greater degree of privilege reduction. For the most permissive filter (e.g. the 'worst case'), it would be a privilege reduction of 37.0% compared to a application-wide filter.

---
\* Since these numbers were derived by dynamic analysis, 49 is likely an underestimate. Static analysis would be needed to generate a usable filter.